# Intro to FITS files for Non-Observers

The vast majority of astronomical images are output as FITS (Flexible Image Transport System) files, and have been for decades. This is one of those rare cases in astronomy where something stuck around because it was objectively an effective system, not just because it worked at the time and then fossilized into the institutional bedrock.

The standards of FITS formatting are maintained and recorded here, if you want to read more or have a reference on hand: https://fits.gsfc.nasa.gov/fits_home.html

The basic unit of a FITS fits is creatively called the HDU (Header Data Unit), consisting of the:
* Data: Usually an image, but may also be a cube, 1D spectrum, or binary data table
* Header: Metadata, a (potentially very long) series of key-value pairs containing (hopefully) everything you need to know about the image in order to work with it.

Most FITS images only have one HDU, but you can have multiple HDUs packaged in one file. These are called multi-extension FITS or MEFs. Each HDU in a MEF can be accessed by index or sometimes by name (key) in AstroPy. However, many older software packages won't work with MEFs, and DS9 does not appear to be able to let you view extensions other than the first image HDU (although it does let you toggle between views of the image header and the Primary HDU header, if those are different).

In [22]:
import numpy as np
import astropy.io.fits as fits

#Sample Herschel Image (same as upload)
hdu = fits.open('/home/pitts/Documents/postdoc/1342211307/level3/herschel.ia.pal.MapContext/extdPLW/hspireplw145_30pxmp_2039_p4416_1456959472545.fits.gz')
#View file contents
print(hdu.info())

Filename: /home/pitts/Documents/postdoc/1342211307/level3/herschel.ia.pal.MapContext/extdPLW/hspireplw145_30pxmp_2039_p4416_1456959472545.fits.gz
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU     283   ()      
  1  image         1 ImageHDU        49   (1806, 2118)   float64   
  2  flag          1 ImageHDU        25   (1806, 2118)   int16   
  3  coverage      1 ImageHDU        47   (1806, 2118)   float64   
  4  error         1 ImageHDU        47   (1806, 2118)   float64   
  5  History       1 ImageHDU        23   ()      
  6  HistoryScript    1 BinTableHDU     39   7R x 1C   [65A]   
  7  HistoryTasks    1 BinTableHDU     46   1R x 4C   [1K, 6A, 1K, 9A]   
  8  HistoryParameters    1 BinTableHDU     74   3R x 10C   [1K, 10A, 7A, 14A, 1L, 1K, 1L, 38A, 4A, 0A]   
None


Header and data of any given HDU may be accessed the way one normally accesses python variable attributes, that is by adding .header or .data like so:

In [2]:
hdr = hdu['image'].header
dat = hdu['image'].data
hdr0 = hdu[0].header 
#because Herschel data traded convenience for compactness and put some 
# of the data we might need for the image in the PrimaryHDU header

Note that HDUs are indexed even if there's only one in the file. Unless you do something like hdu=fits.open('filename')[0], you can never just type HDU.data or HDU.header.

## Units and Coordinates

The most important things that FITS headers tell observers are:
* The units of the data
* How the data maps onto the sky

These data are stored in key-value pairs, where the keys are called "cards". The values are accessed the same way you access entries in a python dictionary, except that header cards are *not* case-sensitive.

In [23]:
print(hdr['bunit'])

MJy/sr


Note that images may have many, many different possible intensity units, some of which require additional parameters. E.g. in MIR, FIR, submm, and radio images, Jy/beam or mJy/beam are common units, which require the cards 'BMAJ', 'BMIN', and 'BPA' to respectively specify the FWHM of the major and minor axes and position angle of the beam/PSF if it is not radially symmetric.

The brilliant minds behind AstroPy (not being sarcastic) also created a way to let us call groups of header cards by glob pattern:

In [8]:
print(hdr['cd*'])

CDELT1  =   -0.003888888888889 / [] WCS: Pixel scale axis 1, unit=Angle         CDELT2  =    0.003888888888889 / [] WCS: Pixel scale axis 2, unit=Angle         END                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     

That brings me to one of the most important aspects of navigating FITS headers, and one of the greatest sources of frustration when having to work with data from different telescopes: specification of a World Coordinate System (WCS)

For a FITS image to correctly encode the position, scale, and orientation of the target on the sky, the following information must be in the header in some form:
1. The WCS reference frame (i.e. are you working in RA/Dec, GLAT/GLON, etc?), including epoch/equinox
2. The WCS projection method (since the plane of the sky is spherical, not flat)
3. The position of the reference point in pixel coordinates
4. The position of the reference point in sky coordinates
5. The angular height and width of each pixel (a.k.a. the Pixel Scale)
6. The rotation angle of the image relative to North in your WCS reference frame
7. The specs for any distortion or warping of the image plane due to the optics
8. Units, reference pixel(s), reference position(s), pixel scale(s), and any distortion parameters for any additional axes

The WCS reference frame and projection method are usually recorded in the same set of keywords, 'CTYPE1' and 'CTYPE2', with the projection method noted after the unit and a few of hyphens. The header should also have the epoch or equinox recorded with the 'epoch' or 'equinox' card.

In [42]:
print(hdr['ctype*'])
print('Epoch = ', hdr['epoch'] if 'epoch' in hdr.cards else hdr['equinox'])

CTYPE1  = 'RA---ARC'           / WCS: Projection type axis 1, default="LINEAR"  CTYPE2  = 'DEC--ARC'           / WCS: Projection type axis 2, default="LINEAR"  END                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     

The equitorial system in particular has 3 variants, any of which can be used with any epoch: ICRS, FK5, and FK4. FK5 and ICRS are the main ones in use today, and while the discrepancies are pretty small, they can matter at high resolutions. Thus, the header should specify which RA/Dec system was used with the 'RADESYS' card.

I'm not going to cover projection methods because I've never had to work with images large enough for them to matter, but the most common ones I've seen are gnomonic/tangent (TAN), orthographic (SIN), Cartesian (CAR), and this one, zenithal/azimuthal equidistant (ARC). A fuller list of supported projections can be found here: https://danmoser.github.io/notes/gai_fits-imgs.html

The position of the reference point in pixel coordinates is encoded in the header cards 'CRPIX1' and 'CRPIX2'. Note that this reference position can be a float. A reference position near the center of an image is often preferred to minimize inaccuracies due to distortions.

The position of the reference pixel in sky coordinates is encoded in the header cards 'CRVAL1' and 'CRVAL2'. Note that the sky coordinate position is usually given in units of degrees.

In [24]:
print(hdr['crpix*'])
print(hdr['crval*'])

CRPIX1  =                903.0 / [] WCS: Reference pixel position axis 1, uni&  CRPIX2  =                  0.0 / [] WCS: Reference pixel position axis 2, uni&  END                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     

There are several different ways to specify the pixel dimensions, rotation angle, and distortions, and sometimes you can't keep them all separate.

The oldest and most common method is to encode the pixel dimensions in 'CDELTi' cards, the rotation as 'CROTA2', and distortion parameters in cards that depend on the form of the distortion model (PC matrices are the only ones I've encountered so far).

Another form is to use a CD matrix, which combines the rotation and pixel dimensions in ways that allow pixels to have shear.

In [21]:
print(hdr['CD*'] if 'CD1_1' in hdr.cards else [hdr['cdelt*'], hdr['crota2']])

[CDELT1  =   -0.003888888888889 / [] WCS: Pixel scale axis 1, unit=Angle         
CDELT2  =    0.003888888888889 / [] WCS: Pixel scale axis 2, unit=Angle         , -22.267708550513095]


Some software packages prefer CD matrices while others favor the older notation. Converting between them is tedious but not difficult. The equations can be found here on pages 4 and 5: https://www.cfa.harvard.edu/~jzhao/SMA-FITS-CASA/docs/wcs88.pdf

If your image has 3+ axes, additional axes will be specified similarly to the first 2 axes. For example, let's say you have an image of HCO+ emission from a handful of clouds at velocities relative to the Local Standard of Rest (v_lsr) for 8 slices from -32 to +7 km/s (inclusive), each slice integrated over 5 km/s. Let's further say that your reference velocity is 0 km/s. Then the parameters needed to describe the third axis might look like this:
>NAXIS3 = 8
>
>CTYPE3 = 'VELO'
>
>CUNIT3 = 'km/s'
>
>CRPIX3 = 6.5000000000E+00
>
>CRVAL3 = 0.0000000000E+00   /V_LSR (km/s)
>
>CDELT3 = 5.0000000000E+00   /Velocity width (km/s)

Notice that "CUNIT3" card (and not just the resemblance to incels' favorite slur). CUNIT is not required, but it is helpful to let you or your image's users look up the units of each axis. If there's no such card, though, the .comments attribute of the header can let you search the comments of a given card for, say, units. Be careful though---the index doesn't go where you'd expect:

In [45]:
print(hdr.comments['cdelt1'])

[] WCS: Pixel scale axis 1, unit=Angle


Knowing how to set more complex distortions is, in my limited experience, usually left to the instrument designers. The only time you should touch them is if you have to plot a 2D image or slice of an image, in which case you may have to delete them from a copy of the header, along with any evidence that the image may have had >2 axes, in order for Matplotlib to accept the header copy as a valid axis transformation.

## Dude, Where's My Spex?
FITS headers are a lot like documentation for code, or the markdown language for python notebooks: they're only as good as the writer makes them.

**FITS headers do not write or update themselves.**

Every reduction pipeline has its own idiosyncracies in what data are recorded and under what card. Cards like NAXISi, CRPIXi, CRVALi, and CTYPEi are pretty much always included. However, I've seen *many* other essential header cards forgotten. Fo instance, ideally, all observatories would record the size of their spatial resolution elements (distinct from pixels, of which each resolution element should contain a minimum of 2 for Nyquist sampling, and ideally more like 3-5) in the headers along with everything else. In practice, only radio observatories generally seem to be good about doing that, which is super annoying because you need that information for almost any kind of flux calculation. Same for the BUNIT card.

Fortunately, most observatories at least make their resolutions and standard flux units readily available and searchable online. All you have to do is add it to your image header. Adding header cards is easy, and can be done one of two ways.

In [48]:
# Check if beam parameters are in header
'BPA' in hdr.cards

False

In [46]:
hdr.set('bmaj',35.4,'unit = arcsec', after='equinox')
hdr.set('bmin',31.2,'unit = arcsec', after='bmaj')
hdr.set('bpa',63.0,'unit = deg', after='bmin')
# Don't quote me on these values; I'm working from memory
# SPIRE images are within 10% of circular, so you could just set BMIN=BMAJ and BPA=0.
# I'm doing otherwise just to make a point.

In [47]:
hdr.comments['bmaj']

'unit = arcsec'

The other way to set a header value is just to set *header*['card']=value, like you're adding a key-value pair to a dictionary called *header*. This method is good for modifying existing header values in-place, but not so good for adding new cards because they are automatically appended to the end, after comments and history entries. *Header.set()* is better for creating and setting the value of a new card because you can control where in the header it is inserted with the kwargs 'before' or 'after'.

In [40]:
newhdu=fits.PrimaryHDU(data=dat[:900,:900])
newhdulist = fits.HDUList([newhdu])
print(newhdulist.info())

Filename: (No file associated with this HDUList)
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU       6   (900, 900)   float64   
None


In [41]:
newhdu.header

SIMPLE  =                    T / conforms to FITS standard                      
BITPIX  =                  -64 / array data type                                
NAXIS   =                    2 / number of array dimensions                     
NAXIS1  =                  900                                                  
NAXIS2  =                  900                                                  
EXTEND  =                    T                                                  